# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
# Import necessary libraries
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import Tool, tool
from lib.parsers import PydanticOutputParser

In [ ]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY") or OPENAI_API_KEY

# Verify API keys are loaded
assert OPENAI_API_KEY is not None, "OPENAI_API_KEY not found in environment variables"
assert TAVILY_API_KEY is not None, "TAVILY_API_KEY not found in environment variables"

print("✓ Environment variables loaded successfully")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
# Initialize ChromaDB client and collection
chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=CHROMA_OPENAI_API_KEY)
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)

@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most relevant results in the vector DB.
    
    Args:
        query: a question about game industry.
    
    Returns:
        Formatted string with game information. Each result contains:
        - Platform: like Game Boy, Playstation 5, Xbox 360...
        - Name: Name of the Game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
    """
    # Query the vector database
    results = collection.query(
        query_texts=[query],
        n_results=5,
        include=['documents', 'metadatas', 'distances']
    )
    
    if not results['documents'][0]:
        return "No games found matching the query."
    
    # Format results
    formatted_results = []
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ), 1):
        result_str = f"Result {i} (relevance score: {1-distance:.4f}):\n"
        result_str += f"  Name: {metadata['Name']}\n"
        result_str += f"  Platform: {metadata['Platform']}\n"
        result_str += f"  Year of Release: {metadata['YearOfRelease']}\n"
        result_str += f"  Genre: {metadata.get('Genre', 'N/A')}\n"
        result_str += f"  Publisher: {metadata.get('Publisher', 'N/A')}\n"
        result_str += f"  Description: {metadata['Description']}\n"
        formatted_results.append(result_str)
    
    return "\n".join(formatted_results)

print("✓ retrieve_game tool created")

#### Evaluate Retrieval Tool

In [ ]:
# Define EvaluationReport model for structured output
class EvaluationReport(BaseModel):
    """Evaluation result for retrieved documents"""
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    description: str = Field(description="Detailed explanation about the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question.
    
    Args:
        question: original question from user
        retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    
    Returns:
        Evaluation result with 'useful' boolean and 'description' explanation.
        The result includes:
        - useful: whether the documents are useful to answer the question
        - description: description about the evaluation result
    """
    # Use LLM as judge to evaluate retrieval quality
    judge_llm = LLM(model="gpt-4o-mini", temperature=0.0)
    
    evaluation_prompt = f"""Your task is to evaluate if the documents are enough to respond to the query.

Question: {question}

Retrieved Documents:
{retrieved_docs}

Evaluate whether these documents contain sufficient information to answer the question accurately.
Give a detailed explanation, so it's possible to take an action to accept it or not.

Respond with a JSON object containing:
- useful: true if the documents are sufficient, false otherwise
- description: a detailed explanation of your evaluation
"""
    
    try:
        # Get structured output using response_format
        response = judge_llm.invoke(
            evaluation_prompt,
            response_format=EvaluationReport
        )
        
        # Parse the response - the content should be JSON
        parser = PydanticOutputParser(model_class=EvaluationReport)
        evaluation = parser.parse(response)
        
        # Format result
        result = {
            "useful": evaluation.useful,
            "description": evaluation.description
        }
    except Exception as e:
        # Fallback: try to parse as JSON directly
        try:
            response = judge_llm.invoke(evaluation_prompt)
            content = response.content
            # Try to extract JSON from the content
            if "{" in content and "}" in content:
                start = content.find("{")
                end = content.rfind("}") + 1
                json_str = content[start:end]
                result = json.loads(json_str)
            else:
                # Fallback evaluation
                result = {
                    "useful": False,
                    "description": f"Could not parse evaluation. Error: {str(e)}"
                }
        except:
            result = {
                "useful": False,
                "description": f"Evaluation failed: {str(e)}"
            }
    
    return json.dumps(result, indent=2)

print("✓ evaluate_retrieval tool created")

#### Game Web Search Tool

In [ ]:
# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

@tool
def game_web_search(question: str) -> str:
    """
    Performs web search to find information about video games when internal 
    knowledge is insufficient.
    
    Args:
        question: a question about game industry.
    
    Returns:
        Search results with relevant information and source citations.
    """
    try:
        # Perform web search using Tavily
        response = tavily_client.search(
            query=question,
            search_depth="advanced",
            max_results=5
        )
        
        # Format results with citations
        formatted_results = []
        formatted_results.append(f"Web search results for: {question}\n")
        
        if 'results' in response:
            for i, result in enumerate(response['results'], 1):
                formatted_results.append(f"Result {i}:")
                formatted_results.append(f"  Title: {result.get('title', 'N/A')}")
                formatted_results.append(f"  URL: {result.get('url', 'N/A')}")
                formatted_results.append(f"  Content: {result.get('content', 'N/A')[:500]}...")
                formatted_results.append("")
        
        # Include answer if available
        if 'answer' in response and response['answer']:
            formatted_results.append(f"\nSummary Answer: {response['answer']}")
        
        return "\n".join(formatted_results)
    
    except Exception as e:
        return f"Error performing web search: {str(e)}"

print("✓ game_web_search tool created")

### Agent

In [ ]:
# Create agent with system instructions and tools
system_instructions = """You are UdaPlay, an AI Research Agent specialized in answering questions about video games.

Your workflow:
1. First, try to answer using internal knowledge by calling retrieve_game with the user's question
2. Evaluate the retrieved results using evaluate_retrieval to assess if they're sufficient
3. If the evaluation indicates the results are not useful or insufficient, use game_web_search to find additional information
4. Combine information from all sources to provide a comprehensive answer
5. Always cite your sources (internal database or web sources)

Guidelines:
- Be accurate and factual
- Cite sources in your responses
- If information is not available, clearly state that
- Provide structured, readable answers
- When using web search results, include URLs as citations

Tools available:
- retrieve_game: Search internal game database
- evaluate_retrieval: Assess if retrieved documents are sufficient
- game_web_search: Search the web for additional information
"""

# Create tools list
tools = [
    Tool.from_func(retrieve_game),
    Tool.from_func(evaluate_retrieval),
    Tool.from_func(game_web_search)
]

# Create agent
agent = Agent(
    model_name="gpt-4o-mini",
    instructions=system_instructions,
    tools=tools,
    temperature=0.7
)

print("✓ Agent created with all tools")

In [ ]:
# Test Query 1: "When was Pokémon Gold and Silver released?"
query1 = "When was Pokémon Gold and Silver released?"

print("=" * 80)
print(f"Query 1: {query1}")
print("=" * 80)

run1 = agent.invoke(query1)
final_state1 = run1.get_final_state()

# Extract final answer
final_messages = final_state1.get("messages", [])
for msg in reversed(final_messages):
    if isinstance(msg, AIMessage) and msg.content:
        print(f"\nAnswer: {msg.content}")
        break

print(f"\nTotal tokens used: {final_state1.get('total_tokens', 0)}")
print(f"Number of steps: {len(run1.snapshots)}")

### (Optional) Advanced

In [ ]:
# Optional: Update your agent with long-term memory
# The agent already uses ShortTermMemory for conversation state
# For long-term memory, you could use the LongTermMemory class from lib.memory
# to store useful information from web searches for future use

# Example of how to add long-term memory:
# from lib.memory import LongTermMemory, MemoryFragment, VectorStoreManager
# from lib.vector_db import VectorStoreManager
# 
# memory_manager = VectorStoreManager(OPENAI_API_KEY)
# long_term_memory = LongTermMemory(memory_manager)
# 
# # After a web search, you could store useful information:
# # memory_fragment = MemoryFragment(
# #     content="Useful game information from web search",
# #     owner="user_id",
# #     namespace="game_info"
# # )
# # long_term_memory.register(memory_fragment)

print("Note: Long-term memory implementation is optional and can be added as needed.")


In [ ]:
# Test Query 2: "Which one was the first 3D platformer Mario game?"
query2 = "Which one was the first 3D platformer Mario game?"

print("=" * 80)
print(f"Query 2: {query2}")
print("=" * 80)

run2 = agent.invoke(query2)
final_state2 = run2.get_final_state()

# Extract final answer
final_messages = final_state2.get("messages", [])
for msg in reversed(final_messages):
    if isinstance(msg, AIMessage) and msg.content:
        print(f"\nAnswer: {msg.content}")
        break

print(f"\nTotal tokens used: {final_state2.get('total_tokens', 0)}")
print(f"Number of steps: {len(run2.snapshots)}")


In [ ]:
# Test Query 3: "Was Mortal Kombat X released for PlayStation 5?"
query3 = "Was Mortal Kombat X released for PlayStation 5?"

print("=" * 80)
print(f"Query 3: {query3}")
print("=" * 80)

run3 = agent.invoke(query3)
final_state3 = run3.get_final_state()

# Extract final answer
final_messages = final_state3.get("messages", [])
for msg in reversed(final_messages):
    if isinstance(msg, AIMessage) and msg.content:
        print(f"\nAnswer: {msg.content}")
        break

print(f"\nTotal tokens used: {final_state3.get('total_tokens', 0)}")
print(f"Number of steps: {len(run3.snapshots)}")


### Detailed Agent Workflow Analysis

Let's examine the agent's workflow for one of the queries to see how it uses tools:


In [ ]:
# Analyze the workflow for query 1
print("Workflow Analysis for Query 1:")
print("=" * 80)

for i, snapshot in enumerate(run1.snapshots, 1):
    print(f"\nStep {i}: {snapshot.step_id}")
    state = snapshot.state_data
    
    # Show tool calls if any
    messages = state.get("messages", [])
    for msg in messages:
        if isinstance(msg, AIMessage) and msg.tool_calls:
            print(f"  Tool calls made:")
            for tc in msg.tool_calls:
                print(f"    - {tc.function.name}")
                try:
                    args = json.loads(tc.function.arguments)
                    print(f"      Arguments: {args}")
                except:
                    pass
        elif isinstance(msg, ToolMessage):
            print(f"  Tool result from {msg.name}: {msg.content[:200]}...")
    
    # Show final answer if available
    if snapshot.step_id == "__termination__":
        for msg in reversed(messages):
            if isinstance(msg, AIMessage) and msg.content:
                print(f"\n  Final Answer: {msg.content[:300]}...")
                break


### Summary

The UdaPlay agent is now fully implemented with:
- ✓ retrieve_game tool: Searches internal vector database
- ✓ evaluate_retrieval tool: Assesses retrieval quality using LLM judge
- ✓ game_web_search tool: Performs web search using Tavily API
- ✓ Agent with state machine workflow
- ✓ Tested with multiple example queries
- ✓ Structured responses with citations

The agent follows a two-tier retrieval approach:
1. First attempts to answer from internal knowledge (RAG)
2. Evaluates the quality of retrieved results
3. Falls back to web search if needed
4. Combines information from all sources with proper citations
